In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

In [ ]:
with open("preprocessed_data.pkl", "rb") as f:
    data = pickle.load(f)

X_train = data["X_train"]
X_val   = data["X_val"]
X_test  = data["X_test"]
y_train = data["y_train"]
y_val   = data["y_val"]
y_test  = data["y_test"]


# Convert to numpy arrays
X_train_np = X_train.values
X_val_np   = X_val.values
X_test_np  = X_test.values


In [ ]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_np.shape[1],)),
    Dropout(0.2),

    Dense(64, activation='relu'),
    Dropout(0.2),

    Dense(32, activation='relu'),

    Dense(1)
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

model.summary()

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


In [ ]:
history = model.fit(
    X_train_np, y_train,
    validation_data=(X_val_np, y_val),
    epochs=100,
    batch_size=128,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
preds = model.predict(X_test_np).flatten()

mse = mean_squared_error(y_test, preds)
rmse = np.sqrt(mse)

print(f"DNN RMSE: {rmse:.4f}")

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(y_test[:300], label="True")
plt.plot(preds[:300], label="Predicted")
plt.title("DNN Predictions vs True Precipitation")
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(10,4))
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Val loss')
plt.title("Training Loss Curve")
plt.legend()
plt.show()